In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import sklearn
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filename = os.path.join(dirname, filename)
        if "train" in filename:
            X_train_origin = pd.read_csv(filename)
        elif 'test' in filename:
            X_test_origin = pd.read_csv(filename)
        else:
            submission = pd.read_csv(filename)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X_train_origin.head(5)

In [ ]:
X_test_origin.head(5)

In [ ]:
X_train_origin.dtypes

In [ ]:
#data visualization
X_train_origin.describe()
        

In [ ]:
X_train_origin.select_dtypes(include = ['object']).describe()

In [ ]:
#create xtrain, xtest, ytrain
y_train = X_train_origin['Survived']
x_test_id = X_test_origin['PassengerId']
X_train = X_train_origin.drop(['PassengerId','Survived', 'Name', 'Ticket', 'Cabin'], axis = 1)
X_test = X_test_origin.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)

In [ ]:
#preprocessing the data
all_X = pd.concat([X_train, X_test], keys = ['train', 'test'], axis = 0)
#dealing with NaN
variables = list(all_X.columns)
for v in variables:
    print(v, all_X[v].isnull().sum())
all_X.describe()

In [ ]:
all_X.select_dtypes(include = ['object']).describe()

In [ ]:
# fill age with mean, Cabin with "0", embarked with S, Fare with median
X_train['Age'].fillna(30, inplace = True)
X_test['Age'].fillna(30, inplace = True)

# X_train['Cabin'].fillna('0', inplace = True)
# X_test['Cabin'].fillna('0', inplace = True)

X_train['Embarked'].fillna('S', inplace = True)
X_test['Embarked'].fillna('S', inplace = True)

X_train['Fare'].fillna(14.45, inplace = True)
X_test['Fare'].fillna(14.45, inplace = True)

In [ ]:
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum())

In [ ]:
X_train

In [ ]:
X_train.dtypes

In [ ]:
#Encoding the variables
categorical = ['Sex','Embarked']
for c in categorical:
    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(X_train[[c]]).toarray())
    for j in list(enc_df.columns):
        X_train[c + "_" + str(j)] = enc_df[j]
    enc_df = pd.DataFrame(enc.fit_transform(X_test[[c]]).toarray())
    for j in list(enc_df.columns):
        X_test[c + "_" + str(j)] = enc_df[j]
    X_train = X_train.drop([c], axis = 1)
    X_test = X_test.drop([c], axis = 1)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
#Normalzied the dataset
sc_X = preprocessing.StandardScaler()
X_train_standarlized = sc_X.fit_transform(X_train)
X_test_standarlized = sc_X.transform(X_test)

In [ ]:
#feature selection
# features = list(X_train.columns)
# from sklearn.ensemble import RandomForestClassifier
# RF = RandomForestClassifier(n_estimators = 100, bootstrap = False)
# RF.fit(X_train, y_train)
# f_rf = {}
# for i in range(len(features)):
#     f_rf[features[i]] = list(RF.feature_importances_)[i]
# f_rf = sorted(f_rf.items(), key = lambda x : x[1], reverse = True)
# print(f_rf)

In [ ]:
#model selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev
#stratified cross validation
def cross_validation(X, Y, clf, name, K = 5):
    x = X.values
    y = Y.values
    skf = StratifiedKFold(n_splits = K, shuffle = False)
    lst_accu_stratified = []
    for train_index, test_index in skf.split(x,y):
        x_train_fold, x_test_fold = x[train_index], x[test_index] 
        y_train_fold, y_test_fold = y[train_index], y[test_index] 
        clf.fit(x_train_fold, y_train_fold) 
        lst_accu_stratified.append(clf.score(x_test_fold, y_test_fold))
    print(name)
    print('\nMaximum Accuracy That can be obtained from this model is:', 
          max(lst_accu_stratified)*100, '%') 
    print('\nMinimum Accuracy:', 
          min(lst_accu_stratified)*100, '%') 
    print('\nOverall Accuracy:', 
          mean(lst_accu_stratified)*100, '%') 
    print('\nStandard Deviation is:', stdev(lst_accu_stratified))
    print("")
    
#Logistic regression
def LR(X_train, y_train):
    clf = LogisticRegression(random_state = 10)
    cross_validation(X_train, y_train, clf, 'LR')

#support vector machine
def SVM(X_train, y_train):
    clf = svm.SVC(probability = True,kernel = 'rbf')
    cross_validation(X_train, y_train, clf, 'SVM')

#RandomForest
def RF(X_train, y_train):
    clf = RandomForestClassifier(n_estimators = 100, bootstrap = False,random_state = 10)
    cross_validation(X_train, y_train, clf, 'RF')

#Decision tree
def DT(X_train, y_train):
    clf = DecisionTreeClassifier(random_state = 10)
    cross_validation(X_train, y_train, clf, 'DT')

#KNN
def KNN(X_train, y_train):
    clf = KNeighborsClassifier(n_neighbors = int(len(X_train)**0.5), p = 2, metric = 'euclidean')
    cross_validation(X_train, y_train, clf, 'KNN')

In [ ]:
LR(X_train, y_train)
RF(X_train, y_train)
SVM(X_train, y_train)
DT(X_train, y_train)
KNN(X_train, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
#grid search for optimization
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

clf = RandomForestClassifier(random_state = 1)

gridF = GridSearchCV(clf, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)


In [ ]:
#Prediction
clf = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, min_samples_leaf = 2, max_depth = 15, bootstrap = False,random_state = 10)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [ ]:
#submission file
submission = pd.DataFrame({'PassengerId': x_test_id,
                           'Survived': predictions})
submission.to_csv('submission.csv', index = False)

In [ ]:
print(gridF.best_params_) 